# Produktion und Distribution

## Das Entscheidungsproblem

In diesem Beispiel werden wir ein einfaches Optimierungsmodell für ein Produktions- und Distributionsproblem formulieren. Wir stellen uns vor, dass wir ein Unternehmen beraten, das Backsteine herstellt. Das Unternehmen hat Produktionsstätten in Baltimore, Cleveland, Little Rock, Birmingham und Charleston. Die Produktion aus diesen Standorten wird an Vertriebszentren in Columbia, Indianapolis, Lexington, Nashville, Richmond und St. Louis geliefert.

Die Marketingabteilung des Unternehmens erstellt laufend Prognosen für die Nachfrage in jedem Vertriebszentrum und das Unternehmen versucht, seine Lieferkette so zu optimieren, dass die Nachfrage in jedem Vertriebszentrum erfüllt wird, während die Gesamtkosten für die Lieferkette minimiert werden. 

Die Transportkosten zwischen den Produktionsstätten und den Vertriebszentren sind bekannt und in @tbl-prod-distr-costs aufgeführt. Die prognostizierte Nachfrage in jedem Vertriebszentrum ist in @tbl-prod-distr-demand aufgeführt. 

Ferner sind die Produktionskapazitäten in den Produktionsstätten begrenzt, wie in @tbl-prod-distr-capacities angegeben. Ausserdem müssen aus technischen Gründen alle Produktionsstätten mindestens 70 Prozent ihrer maximalen Kapazität nutzen.

In [2]:
%pip install gurobipy

# Zuerst importieren wir pandas und gurobipy

import pandas as pd
import gurobipy as gp
from gurobipy import GRB

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Dan laden wir den Datensatz mit den Transportkosten
transp_cost = pd.read_csv("https://raw.githubusercontent.com/febse/data/main/opt/tp_costs1.csv")

# Und lassen uns die ersten paar Zeilen ausdrucken
transp_cost.head()

production  distribution  cost
0  Baltimore      Columbia  4.50
1  Baltimore  Indianapolis  5.09
2  Baltimore     Lexington  4.33
3  Baltimore     Nashville  5.96
4  Baltimore      Richmond  1.96

In [4]:
#| label: tbl-prod-distr-costs
#| tbl-cap: "Transportkosten in EUR pro Einheit zwischen Produktionsstätten und Vertriebszentren"

# Es ist leichter, wenn wir uns die Transportkosten in einer Tabelle ansehen, 
# in der die Zeilen die Produktionsstätten und die Spalten die Vertriebszentren sind.

transp_cost.pivot(index='production', columns='distribution', values='cost')

distribution  Columbia  Indianapolis  Lexington  Nashville  Richmond  \
production                                                             
Baltimore         4.50          5.09       4.33       5.96      1.96   
Birmingham        3.33          4.33       3.38       1.53      5.95   
Charleston        3.02          2.61       1.61       4.44      2.36   
Cleveland         2.43          2.37       2.54       4.13      3.20   
Little Rock       6.42          4.83       3.39       4.40      7.44   

distribution  St. Louis  
production               
Baltimore          7.30  
Birmingham         4.01  
Charleston         4.60  
Cleveland          4.88  
Little Rock        2.92

In [5]:
# Es wird einfacher sein (allerdings nicht notwendig), wenn wir
# den DataFrame mit den Kosten nach Produktionsstätten und Vertriebszentren
# indizieren. Das machen wir mit der Methode set_index.

transp_cost.set_index(['production', 'distribution'], inplace=True)

In [6]:
#| label: tbl-prod-distr-demand

# Hier werden wir die Prognose für die Nachfrage in den Vertriebszentren
# eingeben

distribution = transp_cost.index.levels[1]
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand") 

n_demand.to_frame()

demand
distribution        
Columbia          89
Indianapolis      95
Lexington        121
Nashville        101
Richmond         116
St. Louis        181

In [7]:
#| label: tbl-prod-distr-capacities
#| tbl-cap: "Kapazitäten der Produktionsstätten"

production = transp_cost.index.levels[0]
max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")

max_prod.to_frame()

max_production
production                 
Baltimore               180
Birmingham              200
Charleston              140
Cleveland                80
Little Rock             180


## Das mathematische Modell

Jedes Optimierungsmodell besteht aus vier Komponenten

- Parameter
- Zielfunktion
- Entscheidungsvariablen
- Einschränkungen

In unserem Fall sind die Parameter die Transportkosten, die Nachfrage und die Produktionskapazitäten. Die Zielfunktion besteht darin, die Gesamtkosten zu minimieren. Die Entscheidungsvariablen sind die Anzahl der Einheiten, die von jedem Produktionsstandort an jedes Vertriebszentrum geliefert werden. Die Einschränkungen bestehen darin, dass die Nachfrage in jedem Vertriebszentrum erfüllt wird und dass die Produktionskapazitäten nicht überschritten werden.


| Prod./Distr | Columbia        |   Indianapolis    |    Lexington    | Nashville       | Richmond          |    St. Louis    |
| ----------- | --------------- | :---------------: | :-------------: | --------------- | ----------------- | :-------------: |
| Baltimore   | $x_{balt,col}$  | $x_{balt, indi}$  | $x_{balt,lex}$  | $x_{balt,lex}$  | $x_{balt, rich}$  | $x_{balt,stl}$  |
| Birmingham  | $x_{birm,col}$  | $x_{birm, indi}$  | $x_{birm,lex}$  | $x_{birm,lex}$  | $x_{birm, rich}$  | $x_{birm,stl}$  |
| Charleston  | $x_{charl,col}$ | $x_{charl, indi}$ | $x_{charl,lex}$ | $x_{charl,lex}$ | $x_{charl, rich}$ | $x_{charl,stl}$ |
| Cleveland   | $x_{clev,col}$  | $x_{clev, indi}$  | $x_{clev,lex}$  | $x_{clev,lex}$  | $x_{clev, rich}$  | $x_{clev,stl}$  |
| Little Rock | $x_{littl,col}$ | $x_{littl, indi}$ | $x_{littl,lex}$ | $x_{littl,lex}$ | $x_{littl, rich}$ | $x_{littl,stl}$ |


Mit diesen Variablen können wir die Gesamtkosten für den Transport von Backsteinen von den Produktionsstätten zu den Vertriebszentren ausdrücken.

$$
\min \text{Transportkosten} = 4.50 \times x_{balt,col} + 5.09 \times x_{balt,indi} + 4.33 \times x_{balt,lex} + \ldots + 2.92 \times x_{littl,stl}
$$


Da die Funktion der Transportkosten auch in diesem kleinen Beispiel sehr lang wird (viele Variablen), ist es sinnvoll, die Schreibweise kompakter zu machen. Dazu definieren wir zwei Indexmengen:

$$
\begin{align*}
P & = \{\text{Columbia}, \text{Indianapolis}, \text{Lexington}, \text{Nashville}, \text{Richmond}, \text{St. Louis}\} \\
D & = \{\text{Baltimore}, \text{Cleveland}, \text{Little Rock}, \text{Birmingham}, \text{Charleston}\}
\end{align*}
$$

Mit diesen Mengen können wir die Transportkosten als

$$
\min \text{Transportkosten} = \sum_{p \in P} \sum_{d \in D} \text{cost}_{pd} \times x_{pd}
$$

schreiben, wobei $\text{cost}_{pd}$ die Transportkosten von Produktionsstätte $d$ nach Vertriebszentrum $p$ sind und $x_{pd}$ die Anzahl der Einheiten ist, die von Produktionsstätte $d$ nach Vertriebszentrum $p$ geliefert werden.

Mit dieser Schreibweise können wir auch die Einschränkungen für die Produktionskapazitäten und die Nachfrage in den Vertriebszentren kompakter formulieren. Die Produktionskapazitäten können als

$$
\sum_{p \in P} x_{pd} \leq \text{capacity}_d \quad \forall d \in D
$$

und die Nachfrage in den Vertriebszentren als

$$
\sum_{d \in D} x_{pd} \geq \text{demand}_p \quad \forall p \in P
$$

geschrieben werden.

## Implementierung

In [8]:
# Zuerst reduzieren wir den DataFrame mit den Transportkosten auf eine Serie

transp_cost = transp_cost.squeeze()
print(type(transp_cost))
transp_cost

<class 'pandas.core.series.Series'>


production   distribution
Baltimore    Columbia        4.50
             Indianapolis    5.09
             Lexington       4.33
             Nashville       5.96
             Richmond        1.96
             St. Louis       7.30
Cleveland    Columbia        2.43
             Indianapolis    2.37
             Lexington       2.54
             Nashville       4.13
             Richmond        3.20
             St. Louis       4.88
Little Rock  Columbia        6.42
             Indianapolis    4.83
             Lexington       3.39
             Nashville       4.40
             Richmond        7.44
             St. Louis       2.92
Birmingham   Columbia        3.33
             Indianapolis    4.33
             Lexington       3.38
             Nashville       1.53
             Richmond        5.95
             St. Louis       4.01
Charleston   Columbia        3.02
             Indianapolis    2.61
             Lexington       1.61
             Nashville       4.44
             Richmond 

### Entscheidungsvariablen hinzufügen

Die Entscheidungsvariablen sind die Variablen, über die wir die Kontrolle haben und die der Optimierungslöser (solver) bestimmt.

In der Optimierung gibt es verschiedene Arten von Variablen:
- `Continuous`: Z.B. die Menge eines Produkts, die produziert wird
- `Integer`: Z.B. die Anzahl von Maschinen, die in einer Fabrik installiert werden
- `Binary`: Z.B. die Entscheidung, ob eine Maschine installiert wird oder nicht

Die Entscheidungsvariablen (und Parameter) werden mit Elementen von Mengen indiziert, die wir für das Problem definieren. In diesem Beispiel beginnen wir mit einer Menge von Städten, die unser Produkt herstellen, die wir für die Formulierung als Menge $P$ bezeichnen, aber im Code als 'production' definieren können. Die zweite Indexmenge in unserem Problem sind die Verteilungsorte, die wir in dem mathematischen Modell als die Menge $D$ bezeichnen. In dem Code nennen wir sie 'distribution'. 

Let $x_{p,d}$ be the number of widgets that are produced at facility $p$ and shipped to location $d$. In `gurobipy` gibt es zwei Möglichkeiten, Variablen hinzuzufügen:

- [addVar()](https://www.gurobi.com/documentation/10.0/refman/py_model_addvar.html) fügt eine einzelne Variable hinzu
- [addVars()](https://www.gurobi.com/documentation/10.0/refman/py_model_addvar.html) fügt eine Liste von Variablen hinzu, die durch eine Liste von Indizes indiziert sind.


In [9]:
# Zuerst erstellen wir ein Modell
m = gp.Model("Produktion und Distribution")

# Hier erstellen wir eine leere Liste, in der wir die Variablen für die Routen speichern
routes = []

# Nun iterieren über die Produktionsstätten und die Vertriebszentren
for p in production:
    for d in distribution:
        print(f"Adding a variable for route {p} -> {d}")
        x = m.addVar(name=f"route_{p}_{d}", vtype=GRB.CONTINUOUS)
        routes.append(x)

Restricted license - for non-production use only - expires 2025-11-24
Adding a variable for route Baltimore -> Columbia
Adding a variable for route Baltimore -> Indianapolis
Adding a variable for route Baltimore -> Lexington
Adding a variable for route Baltimore -> Nashville
Adding a variable for route Baltimore -> Richmond
Adding a variable for route Baltimore -> St. Louis
Adding a variable for route Birmingham -> Columbia
Adding a variable for route Birmingham -> Indianapolis
Adding a variable for route Birmingham -> Lexington
Adding a variable for route Birmingham -> Nashville
Adding a variable for route Birmingham -> Richmond
Adding a variable for route Birmingham -> St. Louis
Adding a variable for route Charleston -> Columbia
Adding a variable for route Charleston -> Indianapolis
Adding a variable for route Charleston -> Lexington
Adding a variable for route Charleston -> Nashville
Adding a variable for route Charleston -> Richmond
Adding a variable for route Charleston -> St. Lou

In [10]:
# Eine andere Möglichkeit, die Variablen hinzuzufügen, ist es, Listen als ein Indices zu verwenden
m = gp.Model('Produktion und Distribution')

routes = m.addVars(production, distribution, name="route", vtype=GRB.CONTINUOUS)

In [11]:
# Da wir bereits ein Index für die Produktionsstätten und die Vertriebszentren haben,
# können den Index des DataFrames transp_cost als Index für die Variablen verwenden

m = gp.Model("Produktion und Distribution")

routes = m.addVars(transp_cost.index, name="route", vtype=GRB.CONTINUOUS)

### Einschränkungen hinzufügen

Es gibt zwei Art und Weisen, Einschränkungen zu einem Modell hinzuzufügen:

- [addConstr()](https://www.gurobi.com/documentation/10.0/refman/py_model_addconstr.html) fügt eine einzelne Einschränkung hinzu
- [addConstrs()](https://www.gurobi.com/documentation/10.0/refman/py_model_addconstrs.htmll) fügt eine Liste von Einschränkungen hinzu.

This will be the first time we use [gp.quicksum()](https://www.gurobi.com/documentation/10.0/refman/py_quicksum.html). There are other ways to sum expressions in gurobipy and while this method isn't the most concise to code, it is easy to compare it to the summation in the formulation to see how it works. 

In [12]:
# Die Einschränkungen für die Nachfrage in den Vertriebszentren

meet_demand = m.addConstrs((gp.quicksum((routes[p, d] for p in production)) >= n_demand[d] for d in distribution), name="meet_demand")
meet_demand

{'Columbia': <gurobi.Constr *Awaiting Model Update*>,
 'Indianapolis': <gurobi.Constr *Awaiting Model Update*>,
 'Lexington': <gurobi.Constr *Awaiting Model Update*>,
 'Nashville': <gurobi.Constr *Awaiting Model Update*>,
 'Richmond': <gurobi.Constr *Awaiting Model Update*>,
 'St. Louis': <gurobi.Constr *Awaiting Model Update*>}

In [13]:
can_produce = m.addConstrs(
    (gp.quicksum(routes[p, d] for d in distribution) <= max_prod[p] for p in production),
    name="can_produce"
)
print(can_produce)

must_produce = m.addConstrs(
    (
        gp.quicksum(routes[p, d] for d in distribution) >= 0.7 * max_prod[p] for p in production
    ),
    name="must_produce"
)

must_produce

{'Baltimore': <gurobi.Constr *Awaiting Model Update*>, 'Birmingham': <gurobi.Constr *Awaiting Model Update*>, 'Charleston': <gurobi.Constr *Awaiting Model Update*>, 'Cleveland': <gurobi.Constr *Awaiting Model Update*>, 'Little Rock': <gurobi.Constr *Awaiting Model Update*>}


{'Baltimore': <gurobi.Constr *Awaiting Model Update*>,
 'Birmingham': <gurobi.Constr *Awaiting Model Update*>,
 'Charleston': <gurobi.Constr *Awaiting Model Update*>,
 'Cleveland': <gurobi.Constr *Awaiting Model Update*>,
 'Little Rock': <gurobi.Constr *Awaiting Model Update*>}

### Die Zielfunktion hinzufügen

In `gurobipy` können wir die Zielfunktion mit [setObjective()](https://www.gurobi.com/documentation/10.0/refman/py_model_setobjective.html) setzen. Das zweite Argument heißt `sense` und kann entweder `GRB.MINIMIZE` oder `GRB.MAXIMIZE` sein.

In [14]:
m.setObjective(
    gp.quicksum(routes[p, d] * transp_cost[p,d] for p in production for d in distribution),
    GRB.MINIMIZE
)

### Das Modell lösen

Häufig ist es sinnvoll, das Modell vor der Optimierung als eine `.lp`-Datei auszugeben. Dies kann mit [write()](https://www.gurobi.com/documentation/10.0/refman/py_model_write.html) gemacht werden. Danach können wir uns diese Datei anschauen, um zu sehen, ob wird das Modell korrekt formuliert haben.

In [15]:
m.write('produktion_distribution.lp')

### Die Optimierung starten

In [16]:
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-10400F CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16 rows, 30 columns and 90 nonzeros
Model fingerprint: 0xbf1ec383
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s
Presolved: 11 rows, 35 columns, 65 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.561250e+02   0.000000e+00      0s
      10    1.6404900e+03   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.640490000e+03


### Die Lösung ausgeben

Es gibt mehrere Möglichkeiten, auf die Lösung zuzugreifen. Bevor wir die Lösung ausgeben, müssen wir **immer** überprüfen, ob das Modell überhaupt gelöst wurde. Dies kann mit [status](https://www.gurobi.com/documentation/10.0/refman/py_model_status.html) gemacht werden. Wenn das Modell nicht gelöst wurde, können wir die Lösung nicht ausgeben und der Code wird einen Fehler werfen.

In [17]:
# Überprüfen, ob eine optimale Lösung gefunden wurde

if m.status == GRB.OPTIMAL:
    print("Optimale Lösung gefunden")
    print("Optimaler Zielfunktionswert: ", m.objVal)
else:
    print("Keine optimale Lösung gefunden")

Optimale Lösung gefunden
Optimaler Zielfunktionswert:  1640.4899999999998


In [18]:
[v.x for v in m.getVars()]

[9.999999999999986,
 0.0,
 0.0,
 0.0,
 116.0,
 0.0,
 4.0,
 76.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 180.0,
 75.00000000000001,
 0.0,
 0.0,
 101.0,
 0.0,
 1.0,
 0.0,
 19.0,
 121.0,
 0.0,
 0.0,
 0.0]

In [19]:
routes_sol = pd.DataFrame([v.x for v in m.getVars()], index=transp_cost.index, columns=['value'])
routes_sol.pivot_table(index='production', columns='distribution', values='value')

distribution  Columbia  Indianapolis  Lexington  Nashville  Richmond  \
production                                                             
Baltimore         10.0           0.0        0.0        0.0     116.0   
Birmingham        75.0           0.0        0.0      101.0       0.0   
Charleston         0.0          19.0      121.0        0.0       0.0   
Cleveland          4.0          76.0        0.0        0.0       0.0   
Little Rock        0.0           0.0        0.0        0.0       0.0   

distribution  St. Louis  
production               
Baltimore           0.0  
Birmingham          1.0  
Charleston          0.0  
Cleveland           0.0  
Little Rock       180.0

### Analyse der Lösung

Nun können wir verschiedene Fragen anhand der Lösung beantworten. Die Manager des Unternehmens könnten uns z.B. fragen, wie viele Backsteine in jeder Produktionsstätte produziert werden sollen und wie viele von der maximalen Kapazität der Produktionsstätten genutzt werden.

In [20]:
# Sum the shipment amount by production facility
production_by_site = routes_sol.groupby('production')['value'].sum()

pd.DataFrame({'Remaining': max_prod - production_by_site, 'Utilization': production_by_site/max_prod})

Remaining  Utilization
production                         
Baltimore         54.0        0.700
Birmingham        23.0        0.885
Charleston         0.0        1.000
Cleveland          0.0        1.000
Little Rock        0.0        1.000

Eine andere Art und Weise, die Belastung der Produktionsstätten zu sehen ist bereits in dem optimierten Modell verfügbar.
Als Teil der Optimierung werden `Slack`-Werte für die Kapazitätsbeschränkungen berechnet. Falls diese Werte positiv sind, bedeutet das, dass die Produktionsstätten nicht voll ausgelastet sind. Falls die Werte gleich Null sind, bedeutet das, dass die Einschränkungen genau erfüllt sind.

In [21]:
pd.DataFrame({'Remaining':[can_produce[p].Slack for p in production], 
              'Utilization':[1-can_produce[p].Slack/max_prod[p] for p in production]}, 
             index = production)

Remaining  Utilization
production                         
Baltimore         54.0        0.700
Birmingham        23.0        0.885
Charleston         0.0        1.000
Cleveland          0.0        1.000
Little Rock        0.0        1.000